In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [63]:
df = pd.read_csv('Used/Main Dataset CSV - Gun Laws Subtotaled.csv', header = 0)

In [64]:
df

,Year,Year Code,County,State,County Code,2013 Urbanization,2013 Urbanization Code,Deaths,Population,Crude Rate,...,total_open_carry,total_background_checks,total_assault_large_mags,total_gun_trafficking,total_concealed_carry,total_syg,total_child_access_prev,total_preemption,total_immunity,total_domestic_violence
0,2018,2018,Autauga County,AL,1001,Medium Metro,3,Suppressed,55601,Suppressed,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0
1,2018,2018,Baldwin County,AL,1003,Small Metro,4,26,218022,11.93,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0
2,2018,2018,Barbour County,AL,1005,NonCore (Nonmetro),6,Suppressed,24881,Suppressed,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0
3,2018,2018,Bibb County,AL,1007,Large Fringe Metro,2,Suppressed,22400,Suppressed,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0
4,2018,2018,Blount County,AL,1009,Large Fringe Metro,2,Suppressed,57840,Suppressed,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28318,2010,2010,Sweetwater County,WY,56037,Micropolitan (Nonmetro),5,10,43806,Unreliable,...,3.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0
28319,2010,2010,Teton County,WY,56039,Micropolitan (Nonmetro),5,Suppressed,21294,Suppressed,...,3.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0
28320,2010,2010,Uinta County,WY,56041,Micropolitan (Nonmetro),5,Suppressed,21118,Suppressed,...,3.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0
28321,2010,2010,Washakie County,WY,56043,NonCore (Nonmetro),6,Suppressed,8533,Suppressed,...,3.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,1.0,0.0


In [65]:
df = pd.get_dummies(df, columns = ['2013 Urbanization'], drop_first = True)
df = df.drop(['Year Code','County Code','2013 Urbanization Code'], axis = 1)
df = df.dropna()

df_filtered = df[df['Deaths'] != 'Suppressed']
df_filtered = df_filtered[df_filtered['Deaths'] != 'Missing']
# df.loc[df['Deaths'] == 'Suppressed', 'Deaths'] = 5

df_filtered = df_filtered[df_filtered['Population'] != 'Missing']
#df_filtered = df[df['Population'] != 'Missing']

df_filtered = df_filtered[df_filtered['Poverty %'] != '#VALUE!']
df_filtered['Poverty %'] = pd.to_numeric(df_filtered['Poverty %'])
df_filtered['Deaths'] = pd.to_numeric(df_filtered['Deaths'])
df_filtered['Population'] = pd.to_numeric(df_filtered['Population'])
df_filtered['Crude Rate'] = df_filtered['Deaths'] / (df_filtered['Population'] / 100000)
df_filtered['Education Year'] = [1 if x > 2013 else 0 for x in df_filtered['Year']]

In [66]:
df_filtered['Education Score'] = np.where(df_filtered['Education Year'] == 0, df_filtered['Percent of adults with less than a high school diploma, 2000'] * 1 + df_filtered['Percent of adults with a high school diploma only, 2000'] * 2 + df_filtered["Percent of adults completing some college or associate's degree, 2000"] * 3 + df_filtered["Percent of adults with a bachelor's degree or higher, 2000"] * 4 , df_filtered['Percent of adults with less than a high school diploma, 2014-18'] * 1 + + df_filtered['Percent of adults with a high school diploma only, 2014-18'] * 2 + df_filtered["Percent of adults completing some college or associate's degree, 2014-18"] * 3 + df_filtered["Percent of adults with a bachelor's degree or higher, 2014-18"] * 4)

In [67]:
df_filtered['Deaths'].value_counts()

10     465
11     412
12     339
13     287
15     236
      ... 
721      1
913      1
146      1
178      1
629      1
Name: Deaths, Length: 257, dtype: int64

In [68]:
df_filtered.to_csv('Dataset For Modeling w Death Count Only.csv')

In [69]:
fig, ax = plt.subplots()


,Year,County,State,Deaths,Population,Crude Rate,% Gun Ownership,Poverty %,"Percent of adults with less than a high school diploma, 2000","Percent of adults with a high school diploma only, 2000",...,total_preemption,total_immunity,total_domestic_violence,2013 Urbanization_Large Fringe Metro,2013 Urbanization_Medium Metro,2013 Urbanization_Micropolitan (Nonmetro),2013 Urbanization_NonCore (Nonmetro),2013 Urbanization_Small Metro,Education Year,Education Score
3147,2011,Autauga County,AL,10,55267,18.093980,0.513,0.149,0.213,0.338,...,0.0,0.0,1.0,0,1,0,0,0,0,2.416
3148,2011,Baldwin County,AL,32,186717,17.138236,0.513,0.134,0.180,0.296,...,0.0,0.0,1.0,0,0,0,0,1,0,2.575
3151,2011,Blount County,AL,10,57677,17.337934,0.513,0.149,0.296,0.360,...,0.0,0.0,1.0,1,0,0,0,0,0,2.144
3154,2011,Calhoun County,AL,24,117797,20.374033,0.513,0.209,0.261,0.322,...,0.0,0.0,1.0,0,0,0,0,1,0,2.305
3168,2011,Cullman County,AL,12,80536,14.900169,0.513,0.182,0.296,0.320,...,0.0,0.0,1.0,0,0,1,0,0,0,2.207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28298,2010,Winnebago County,WI,15,166994,8.982359,0.431,0.120,0.137,0.374,...,0.0,1.0,6.0,0,0,0,0,1,0,2.580
28302,2010,Campbell County,WY,11,46133,23.844103,0.572,0.068,0.117,0.353,...,0.0,1.0,0.0,0,0,1,0,0,0,2.570
28310,2010,Laramie County,WY,16,91738,17.440973,0.572,0.120,0.109,0.265,...,0.0,1.0,0.0,0,0,0,0,1,0,2.751
28312,2010,Natrona County,WY,14,75450,18.555335,0.572,0.103,0.117,0.306,...,0.0,1.0,0.0,0,0,0,0,1,0,2.657
